In [1]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

import os
os.chdir('..')

# import the necessary package
from adbench.datasets.data_generator import DataGenerator
from adbench.myutils import Utils

datagenerator = DataGenerator()
utils = Utils()

In [6]:
from adbench.baseline.PyOD import PYOD
from adbench.baseline.DevNet.run import DevNet
from adbench.baseline.Supervised import supervised

# dataset and model list / dict
dataset_list = ['6_cardio']
model_dict = {'IForest': PYOD, 'DevNet': DevNet, 'CatB': supervised}

# save the results
df_AUCROC = pd.DataFrame(data=None, index=dataset_list, columns = model_dict.keys())
df_AUCPR = pd.DataFrame(data=None, index=dataset_list, columns = model_dict.keys())

In [7]:
# seed for reproducible results
seed = 42

for dataset in dataset_list:
    '''
    la: ratio of labeled anomalies, from 0.0 to 1.0
    realistic_synthetic_mode: types of synthetic anomalies, can be local, global, dependency or cluster
    noise_type: inject data noises for testing model robustness, can be duplicated_anomalies, irrelevant_features or label_contamination
    '''
    
    # import the dataset
    datagenerator.dataset = dataset # specify the dataset name
    data = datagenerator.generator(la=0.1, realistic_synthetic_mode=None, noise_type=None) # only 10% labeled anomalies are available
    
    for name, clf in model_dict.items():
        # model initialization
        if name == 'DevNet':
            clf = clf(seed=seed, model_name=name, save_suffix='test') # DevNet use early stopping to save the model parameter
        else:
            clf = clf(seed=seed, model_name=name)
        
        # training, for unsupervised models the y label will be discarded
        clf = clf.fit(X_train=data['X_train'], y_train=data['y_train'])
        
        # output predicted anomaly score on testing set
        score = clf.predict_score(data['X_test'])

        # evaluation
        result = utils.metric(y_true=data['y_test'], y_score=score)
        
        # save results
        df_AUCROC.loc[dataset, name] = result['aucroc']
        df_AUCPR.loc[dataset, name] = result['aucpr']

current noise type: None
{'Samples': 1831, 'Features': 21, 'Anomalies': 176, 'Anomalies Ratio(%)': 9.61}
best param: None
Training size: 1281, No. outliers: 12
Epoch 1/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 103ms/step - loss: 2.7348
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 95ms/step - loss: 2.5912 
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step - loss: 2.5211
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 77ms/step - loss: 2.4440
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 107ms/step - loss: 2.3625
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - loss: 2.2843
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - loss: 2.1985
Epoch 8/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 78ms/step - loss: 2.0900
Epoch 9/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 103ms/step - loss: 2.0051
Epoch 10/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 78ms/step - loss: 1.8929
Epoch 11/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - loss: 1.7962
Epoch 12/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - loss: 1.6777
Epoch 13/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s

In [8]:
df_AUCROC

,IForest,DevNet,CatB
6_cardio,0.944193,0.993167,0.983827


In [9]:
df_AUCPR

,IForest,DevNet,CatB
6_cardio,0.615718,0.95821,0.909086
